In [15]:
import pandas as pd
from ltp import LTP
import re
ltp = LTP()

In [16]:
with open("professions.txt",'r' , encoding='UTF-8') as f:
    profession = f.read()
    profession = profession.split('\n')

def clean (text): # 洗掉英文字符
    text =  str(text)
    temp = re.sub('[a-zA-Z]','',text)
    return temp

def process (text):
    seg, hidden = ltp.seg([ text])
    ner = ltp.ner(hidden)
    
    name = []
    age = []
    place = []
    action = []
    work = []
    p1 = r"男|女"
    p2 = r'岁'
    gender = re.findall( p1, text)
    age = re.findall( p2, text)
    for i in profession:
        if i in text and i != '员':
            work.append(i)
    
    # 正则表达式 提取 年龄 性别
    # 命名实体nh 姓名 ni机构 ns 地点
    # ner[0] 结构为 （标签，标记起始，标记末尾）
    for item in ner[0]:
        tag, start, end = item
        #print(tag,":", "".join(seg[0][start:end + 1]))
        if(tag == 'Nh'): #name 
            name.append("".join(seg[0][start:end + 1]) )
        elif ( tag == 'Ns' or  tag == 'Ni') :
            place.append("".join(seg[0][start:end + 1]) )
    
    # pos 标记动作
    ## 动作标记为'v'
    pos = ltp.pos(hidden)
    for i in range(len(seg[0])):
        if (pos[0][i] == 'v'):
            action.append(seg[0][i] )
    
    # 数据0 1 标记
    #label
    if len(name) == 0:
        name_label = 0
    elif '某' in name:
        name_label = 0.5
    else:
        name_label = 1
        
    age_label = 1 if len(age) != 0 else 0
    gender_label = 1 if len(gender) != 0 else 0
    work_label = 1 if len(work) != 0 else 0
    
    return (name ,name_label, age, age_label ,gender, gender_label,  work, work_label ,place, action )

In [17]:
#打开excel
import pandas as pd
df = pd.read_excel('./Data/Shenzhen.xlsx', engine='openpyxl', header = 0)
df.head(5)

,字段,时间,字段1,字段2,标题,正文,发布时间
0,1,2021-11-18,http://wjw.sz.gov.cn/yqxx/content/post_9378163...,City reports one imported confirmed COVID-19 case,City reports one imported confirmed COVID-19 case,"On Nov. 17, Shenzhen reported one imported con...",发布日期：2021-11-18 14:02:12
1,2,2021-11-18,http://wjw.sz.gov.cn/yqxx/content/post_9377483...,2021年11月18日深圳市新冠肺炎疫情情况,2021年11月18日深圳市新冠肺炎疫情情况,11月17日0-24时，深圳新增1例境外输入确诊病例。 1例境外输入确诊病例为男性，中国籍...,发布日期：2021-11-18 10:46:35
2,3,2021-11-17,http://wjw.sz.gov.cn/yqxx/content/post_9374829...,Updates on COVID-19 cases in Shenzhen (Nov. 17),Updates on COVID-19 cases in Shenzhen (Nov. 17),"On Nov. 16, Shenzhen reported no new confirmed...",发布日期：2021-11-17 15:38:11
3,4,2021-11-17,http://wjw.sz.gov.cn/yqxx/content/post_9373432...,2021年11月17日深圳市新冠肺炎疫情情况,2021年11月17日深圳市新冠肺炎疫情情况,11月16日0-24时，深圳无新增确诊病例和无症状感染者。 截至11月16日24时，深圳市...,发布日期：2021-11-17 11:04:25
4,5,2021-11-16,http://wjw.sz.gov.cn/yqxx/content/post_9372037...,第1针北京科兴，第2针北京生物，加强针打啥？,第1针北京科兴，第2针北京生物，加强针打啥？,国家卫生健康委11月15日发布新冠疫苗加强免疫接种相关问答。问答指出，预防新冠肺炎最好的办法...,发布日期：2021-11-16 21:45:28


In [18]:
df['正文'] = df['正文'].apply(clean) #清除字母

In [19]:
# clean_data_df: [time,content]
clean_data = {'time':df['时间'] ,'content': df['正文']}
clean_data_df = pd.DataFrame(clean_data)
clean_data_df = clean_data_df.groupby(by='time').sum('content') # 按时间合并
print(clean_data_df.head)
clean_data_df.to_excel("./Data/sz_clean.xlsx")

<bound method NDFrame.head of                                                       content
time                                                         
2020-01-20  1月19日，国家卫生健康委确认深圳首例输入性新型冠状病毒感染的肺炎确诊病例。\n　　患者为男...
2020-01-21  1月20日晚，广东省卫健委公布，20日全省共新增13例新型冠状病毒感染的肺炎确诊病例，其中，...
2020-01-22  据广东省卫健委官网通报，截至1月21日24:00——\n\n　　广东累计发现“新型肺炎”确诊...
2020-01-23  根据广东省卫健委官网通报，截至1月22日24时，广东省累计报告新型冠状病毒感染的肺炎确诊病例...
2020-01-24  截至1月23日24时，深圳市累计报告新型冠状病毒感染的肺炎确诊病例15例，危重病例2例，出院...
...                                                       ...
2021-11-14   . 13,        -19. ,     ,      ,     . 27.   ...
2021-11-15   . 14,          -19.    . 14,        588  .　　1...
2021-11-16  国家卫生健康委11月15日发布新冠疫苗加强免疫接种相关问答。问答指出，预防新冠肺炎最好的办法...
2021-11-17   . 16,          -19.    . 16,        588  .　　1...
2021-11-18   . 17,        -19. ,     ,       . 14.        ...

[669 rows x 1 columns]>


In [20]:
# for test
print (clean_data_df['content'][600])
res = process (clean_data_df['content'][600])
for i in res:
    print(i)

 . 10,        -19. ,     ,    ,  ,       . 7.                ’     . ,      .　　9月10日0-24时，深圳新增1例境外输入无症状感染者。　　1例境外输入无症状感染者为中国籍，男性，为肯尼亚输入。9月7日从印尼雅加达乘机抵深，入境后即被集中隔离，后转至深圳市第三人民医院应急院区观察治疗，情况稳定。9月10日0-24时，深圳新增1例境外输入无症状感染者　　1例境外输入无症状感染者为中国籍，男性，为肯尼亚输入。9月7日从印尼雅加达乘机抵深，入境后即被集中隔离，后转至深圳市第三人民医院应急院区观察治疗，情况稳定。深圳市卫生健康委员会2021年9月11日
[]
0
[]
0
['男', '男']
1
[]
0
['深圳', '肯尼亚', '印尼雅加达', '深圳市', '人民医院', '深圳', '肯尼亚', '印尼雅加达', '深圳市', '人民医院', '深圳市', '委员会']
['新增', '输入', '无症状感染者', '输入', '无症状感染者', '为', '输入', '抵深', '入境', '隔离', '转至', '应急院区', '观察', '治疗', '新增', '输入', '无症状感染者', '输入', '无症状感染者', '为', '输入', '抵深', '入境', '隔离', '转至', '应急院区', '观察', '治疗']


In [21]:
# (name ,name_label, age, age_label ,gender, gender_label,  work, work_label ,place, action )

In [22]:
data = pd.read_excel("./Results/sz_cleaned.xlsx",engine='openpyxl',header=0)
data.head(20)
data['content'][3]

selected=[]
time = []

for index, obs in data.iterrows():
    item = re.findall(r'病例\d.*\n\n\u3000\u3000.*。\n',obs['content'],re.I|re.M)
    item_relative = re.findall(r'病例\d\n\n\u3000\u3000病例\d.*\n\n\u3000\u3000.*。\n',obs['content'],re.I|re.M)
    if item == None and item_relative==None:
        continue
    else:   
        if len(item)==1:
            selected.append(item)
            time.append(obs['time'])
        else:
            for i in item:
                selected.append(i)
                time.append(obs['time'])
        if item_relative != None:
            item = item_relative
            if len(item)==1:
                selected.append(item)
                time.append(obs['time'])
            else:
                for i in item:
                    selected.append(i)
                    time.append(obs['time'])
    

Data_select = pd.DataFrame({'time': time, 'content':selected})
Data_select.head(10)
Data_select.to_excel('.\Results\sz_cases.xlsx')
   

In [23]:
name_col = []
name_label_col = []
age_col = []
age_label_col = []
gender_col = []
gender_label_col = []
work_col = []
work_label_col= []
place_col = []
action_col = []

clean_data_df = pd.read_excel("./Results/sz_cases.xlsx",engine='openpyxl',header=0)

In [24]:
for i in clean_data_df['content']:
    name ,name_label, age, age_label ,gender, gender_label,  work, work_label ,place, action = process(i)
    name_col .append(name)
    name_label_col .append(name_label)
    age_col.append(age)
    age_label_col .append(age_label)
    gender_col.append(gender)
    gender_label_col .append(gender_label)
    work_col = [].append(work)
    work_label_col.append(work_label)
    place_col .append(place)
    action_col .append(action)

In [25]:
result_data = {    
            'time' : clean_data_df['time'] , 
            'content': clean_data_df['content'] , 
            'name': name_col,  
            'name_label': name_label_col ,
            'age': age_col,
            'age_label': age_label_col,
            'gender':gender_col,
            'gender_label':gender_label_col ,
            'work': work_col,
            'work_label':work_label_col,
            'place':place_col,
            'action':action_col }
res = pd.DataFrame(result_data)
print(res.head)
res.to_excel("./Results/sz_cases_results.xlsx")

<bound method NDFrame.head of             time                                            content  \
0     2020-01-22  ['病例26例，其中重症7例，危重3例，无死亡病例。\n\n\u3000\u3000深圳市的...   
1     2020-01-23  ['病例32例，其中深圳累计确诊15例。\n\n\u3000\u30001月23日，经省市专...   
2     2020-01-24  病例15例，危重病例2例，出院病例2例，无死亡病例。\n\n　　深圳累计确诊的15个“新冠肺...   
3     2020-01-24  病例1\n\n　　深圳市南山区66岁男性患者，2019年12月29日到武汉探亲，2020年1...   
4     2020-01-24  病例1妻子\n\n　　深圳市南山区65岁女性患者，2019年12月29日到武汉探亲，2020...   
...          ...                                                ...   
1377  2021-06-07  ['病例508例，其中本土病例424例，境外输入病例84例。6月6日0-24时，深圳新增1例...   
1378  2021-06-13  ['病例512例。6月12日0-24时，深圳无新增确诊病例和无症状感染者。\n\n\u300...   
1379  2021-06-20  ['病例38例，其中确诊患者30人（其中7例由无症状感染者转为确诊病例）、无症状感染者8人。...   
1380  2021-08-06  ['病例124例，其中本土病例80例。\n\n\u3000\u30008月5日下午，国务院联...   
1381  2021-09-06  ['病例9月4日从肯尼亚内罗毕乘机抵深，1例病例8月24日从印尼雅加达乘机抵深。以上病例入境...   

              name  name_label  age  age_label  gender  gender_label  work  \
0               []           0   []    